In [1]:
from foods_and_recipes import foods_df, recipes_df, RECIPES_BY_NAME
from nutrition_models import DogProfile, weight_loss_target_kcal, weight_loss_target_kj

oakley = DogProfile(name="Oakley", current_weight_kg=40.0, ideal_weight_kg=35.0)

target_kcal = weight_loss_target_kcal(oakley.ideal_weight_kg)
target_kj = weight_loss_target_kj(oakley.ideal_weight_kg)

df_foods = foods_df()
df_recipes = recipes_df()

base = RECIPES_BY_NAME["Base daily stew"]
base.total_kj, base.total_kcal, base.total_cost, target_kj, target_kcal

(2830.0, 676.3862332695984, 4.125, 3371.5587840990784, 805.8218891250187)

In [8]:
from datetime import date, timedelta

import pandas as pd

from nutrition_models import DailyMealPlan, grocery_list_from_plans
from foods_and_recipes import RECIPES_BY_NAME

# --- 1) Build a weekly meal plan ---

start_day = date.today()

# Example: Mon–Sun all use "Base daily stew".
# You can swap in other recipes per day if you want.
week_plans = []
for i in range(7):
    d = start_day + timedelta(days=i)
    plan = DailyMealPlan(day=d)
    plan.add_recipe(RECIPES_BY_NAME["Base daily stew"])
    week_plans.append(plan)

# --- 2) Build grocery list: FoodItem -> total grams across the week ---

totals = grocery_list_from_plans(week_plans)  # {FoodItem: grams}

# --- 3) Convert to a DataFrame with total weight and cost per food ---

rows = []
for food, grams in totals.items():
    cost = food.cost(grams)
    rows.append(
        {
            "food_name": food.name,
            "category": food.category.name,
            "total_grams": grams,
            "dollars_per_kg": food.dollars_per_kg,
            "total_cost": cost,
        }
    )

df_week_grocery = pd.DataFrame(rows).sort_values("food_name").reset_index(drop=True)

# --- 4) Total cost for the week ---

total_week_cost = df_week_grocery["total_cost"].sum()

styled = df_week_grocery.style.format({
    "total_cost": "${:.2f}",
    "dollars_per_kg": "${:.2f}",
    "total_grams": "{:.0f}",
})

# Style and render the grocery list with per-column formatting.
(
    df_week_grocery
    .style
    .format({
        "total_cost": "${:.2f}",
        "dollars_per_kg": "${:.2f}",
        "total_grams": "{:.0f}",
    })
)

,food_name,category,total_grams,dollars_per_kg,total_cost
0,Cooked brown rice,CARB,1050,$3.00,$3.15
1,Cooked chicken breast,PROTEIN,1750,$12.00,$21.00
2,Fish oil,SUPPLEMENT,35,$30.00,$1.05
3,Pumpkin puree,CARB,350,$2.50,$0.88
4,Steamed mixed veg,VEG,700,$4.00,$2.80


In [9]:
print(f"Total weekly cost: ${total_week_cost:.2f}")

Total weekly cost: $28.88
